<a href="https://colab.research.google.com/github/oluwadamilola/Topic-Modelling/blob/development/Topic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Prep work: Downloading necessary files
Before we get started, we need to download all of the data we'll be using.

singnal financial news dataset -

#import Libraries

In [2]:
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 26.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.2 which is incompatible.
tensorflow 2.12.0 

In [1]:
import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

# The Data Preprocessing

*   cnbc data was used because there is a limited resource on colab to handle large data




In [2]:
import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/cnbc_headlines.csv")

# Clean it up a little bit, removing non-word characters (numbers and ___ etc)
#df.content = df.content.str.replace("[^A-Za-z ]", " ")

data.head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Headlines,Time,Description
0,Jim Cramer: A better way to invest in the Covi...,"7:51 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer recommended buying..."
1,Cramer's lightning round: I would own Teradyne,"7:33 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer rings the lightnin..."
2,NaN,NaN,NaN
3,"Cramer's week ahead: Big week for earnings, ev...","7:25 PM ET Fri, 17 July 2020","""We'll pay more for the earnings of the non-Co..."
4,IQ Capital CEO Keith Bliss says tech and healt...,"4:24 PM ET Fri, 17 July 2020","Keith Bliss, IQ Capital CEO, joins ""Closing Be..."


In [3]:
#drop NAN values
data = data.dropna()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Headlines,Time,Description
0,Jim Cramer: A better way to invest in the Covi...,"7:51 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer recommended buying..."
1,Cramer's lightning round: I would own Teradyne,"7:33 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer rings the lightnin..."
3,"Cramer's week ahead: Big week for earnings, ev...","7:25 PM ET Fri, 17 July 2020","""We'll pay more for the earnings of the non-Co..."
4,IQ Capital CEO Keith Bliss says tech and healt...,"4:24 PM ET Fri, 17 July 2020","Keith Bliss, IQ Capital CEO, joins ""Closing Be..."
5,Wall Street delivered the 'kind of pullback I'...,"7:36 PM ET Thu, 16 July 2020","""Look for the stocks of high-quality companies..."


In [5]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords

# Preprocess the text data
def preprocess_text(text):
    return [word for word in simple_preprocess(text) if word not in gensim.parsing.preprocessing.STOPWORDS]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Tokenize words and cleanup the text

In [6]:
import nltk
nltk.download('stopwords')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:

import nltk
nltk.download('punkt')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#create Dictionary

In [8]:
# Create a dictionary and corpus for the entire dataset
all_words = data['Description'].apply(preprocess_text)
dictionary = gensim.corpora.Dictionary(all_words)
corpus = [dictionary.doc2bow(text) for text in all_words]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)], [(2, 1), (4, 1), (5, 1), (6, 1), (7, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1)], [(1, 1), (2, 1), (10, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1)], [(12, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1)], [(1, 1), (4, 1), (6, 1), (7, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1)], [(2, 1), (4, 1), (5, 1), (6, 1), (7, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1)], [(33, 1), (38, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1)], [(1, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Build LDA Model

In [10]:
from gensim import models
# Running LDA
num_topics = 10  # Specify the desired number of topics
lda_model = gensim.models.LdaModel(
    corpus,
    num_topics=num_topics,
    id2word=dictionary,
    passes=10,
    random_state=42
)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Print the topics and their top words
for topic_id, topic_words in lda_model.print_topics():
    print("Topic", topic_id, ":", topic_words)

# Visualize the Topics

In [11]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.079324  0.113617       1        1  19.054614
3      0.085226  0.115901       2        1  14.038998
2      0.066736 -0.298958       3        1  12.628741
7     -0.024175  0.023596       4        1  12.488794
1      0.136829  0.101418       5        1  11.436639
5      0.210018 -0.064360       6        1   8.725004
9     -0.105211  0.026264       7        1   8.622070
0     -0.085281  0.000485       8        1   5.984172
8     -0.168994  0.022872       9        1   4.311537
4     -0.194472 -0.040834      10        1   2.709429, topic_info=            Term        Freq       Total Category  logprob  loglift
13       callers  360.000000  360.000000  Default  30.0000  30.0000
56        stocks  522.000000  522.000000  Default  29.0000  29.0000
40     including  202.000000  202.000000  Default  28.0000  28.0000
433     favorite  211.000000  211.000000  Default  27.0000  27.0000
15     lightning  199.000000  199.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
4           host    8.863211  290.097911  Topic10  -4.7213   0.1201
256         best    5.334074   49.869263  Topic10  -5.2291   1.3731
37       capital    3.847871   18.308566  Topic10  -5.5557   2.0486
44       markets    4.437568   45.013992  Topic10  -5.4131   1.2916
674  opportunity    3.683667   31.028485  Topic10  -5.5993   1.4774

[630 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1350      1  0.643676  according
1350      7  0.302906  according
4727      4  0.814563   acquired
1517      1  0.849473        act
618       1  0.214434     action
...     ...       ...        ...
5504      6  0.789117     zipped
3563      3  0.227408       zips
3563      6  0.758028       zips
5513      6  0.910127     zoomed
4093      6  0.970083      zooms

[1060 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 4, 3, 8, 2, 6, 10, 1, 9, 5])

#Topic inference

In [12]:
topic_inferences = []

for doc_bow in corpus:
    document_topics = lda_model.get_document_topics(doc_bow)
    topic_inferences.append(document_topics)

# Print topic inferences for each document
for doc_idx, document_topics in enumerate(topic_inferences):
    print(f"Document {doc_idx + 1} - Topic Inference:")
    for topic_id, topic_prob in document_topics:
        print(f"Topic {topic_id} - Probability: {topic_prob:.4f}")
    print()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Streaming output truncated to the last 5000 lines.
Topic 6 - Probability: 0.3227

Document 1909 - Topic Inference:
Topic 5 - Probability: 0.7597
Topic 9 - Probability: 0.1831

Document 1910 - Topic Inference:
Topic 5 - Probability: 0.7186
Topic 8 - Probability: 0.2242

Document 1911 - Topic Inference:
Topic 1 - Probability: 0.6088
Topic 9 - Probability: 0.3340

Document 1912 - Topic Inference:
Topic 6 - Probability: 0.8648
Topic 9 - Probability: 0.0818

Document 1913 - Topic Inference:
Topic 7 - Probability: 0.9181

Document 1914 - Topic Inference:
Topic 5 - Probability: 0.9307

Document 1915 - Topic Inference:
Topic 1 - Probability: 0.3450
Topic 3 - Probability: 0.4883
Topic 8 - Probability: 0.1255

Document 1916 - Topic Inference:
Topic 2 - Probability: 0.1125
Topic 6 - Probability: 0.8147

Document 1917 - Topic Inference:
Topic 6 - Probability: 0.9357

Document 1918 - Topic Inference:
Topic 1 - Probability: 0.8021
Topic 9 - Probability: 0.1363

Document 1919 - Topic Inference:
Topic

In [13]:
# Get the most probable and second most probable topics for each document
most_probable_topics = []
second_most_probable_topics = []
for doc_bow in corpus:
    doc_topics = lda_model.get_document_topics(doc_bow)
    sorted_topics = sorted(doc_topics, key=lambda x: x[1], reverse=True)
    most_probable_topic = sorted_topics[0][0]
    second_most_probable_topic = sorted_topics[1][0] if len(sorted_topics) > 1 else None
    most_probable_topics.append(most_probable_topic)
    second_most_probable_topics.append(second_most_probable_topic)

# Print the most probable and second most probable topics for each document
topic_keywords = lda_model.show_topics(num_topics=num_topics, num_words=15)  # Adjust the number of words as needed

for i in range(len(most_probable_topics)):
    most_probable_topic = most_probable_topics[i]
    second_most_probable_topic = second_most_probable_topics[i]

    most_probable_keywords = [word.strip() for word in re.findall(r'"([^"]*)"', topic_keywords[most_probable_topic][1])]
    second_most_probable_keywords = [word.strip() for word in re.findall(r'"([^"]*)"', topic_keywords[second_most_probable_topic][1])] if second_most_probable_topic is not None else None

    print(f"Document {i+1}:")
    print(f"Most Probable Topic - {most_probable_topic}, Keywords: {most_probable_keywords}")
    print(f"Second Most Probable Topic - {second_most_probable_topic}, Keywords: {second_most_probable_keywords}")
    print()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Streaming output truncated to the last 5000 lines.
Document 1551:
Most Probable Topic - 6, Keywords: ['jim', 'cramer', 'market', 'says', 'stock', 'stocks', 'ceo', 'explains', 'president', 'new', 'sits', 'trump', 'time', 'shares', 'company']
Second Most Probable Topic - 2, Keywords: ['stock', 'jim', 'cramer', 'lightning', 'speed', 'money', 'mad', 'host', 'callers', 'questions', 'round', 'rapid', 'answers', 'means', 'giving']

Document 1552:
Most Probable Topic - 6, Keywords: ['jim', 'cramer', 'market', 'says', 'stock', 'stocks', 'ceo', 'explains', 'president', 'new', 'sits', 'trump', 'time', 'shares', 'company']
Second Most Probable Topic - None, Keywords: None

Document 1553:
Most Probable Topic - 1, Keywords: ['jim', 'cramer', 'investors', 'stocks', 'earnings', 'market', 'explains', 'week', 'ahead', 'stock', 'breaks', 'buying', 'tech', 'shares', 'amazon']
Second Most Probable Topic - 2, Keywords: ['stock', 'jim', 'cramer', 'lightning', 'speed', 'money', 'mad', 'host', 'callers', 'ques

# Get the most probable topics

#Topic Interpretation on Time Series Graph

In [15]:
# Create a function to get topics for each document
def get_document_topics(text):
    bow = dictionary.doc2bow(preprocess_text(text))
    return lda_model.get_document_topics(bow)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
data['Time'] = pd.to_datetime(data['Time'])
data.set_index('Time', inplace=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
<ipython-input-16-c7fc3ebb4ee7>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Time'] = pd.to_datetime(data['Time'])


In [17]:
weekly_data = data.resample('W').agg({'Description': ' '.join})
 #Get topics for each week
weekly_topics = weekly_data['Description'].apply(get_document_topics)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Convert weekly_topics to a DataFrame
topics_df = pd.DataFrame(weekly_topics.tolist(), index=weekly_topics.index)

In [ ]:
# Convert weekly_topics to a DataFrame
topics_df = pd.DataFrame(weekly_topics.tolist(), index=weekly_topics.index)

# Resample the DataFrame to a daily frequency and forward-fill missing values
daily_topics_df = topics_df.resample('D').ffill()

# Drop rows with any remaining missing values
daily_topics_df.dropna(inplace=True)

# Convert tuples to topic probabilities
daily_topics_df = daily_topics_df.applymap(lambda x: x[1])

# Plot each topic's occurrence on a daily basis using a line graph
plt.figure(figsize=(10, 6))
for topic in daily_topics_df.columns:
    plt.plot(daily_topics_df.index, daily_topics_df[topic], label=f"Topic {topic}")

plt.xlabel('Time')
plt.ylabel('Topic Probability')
plt.title('Topic Occurrence on a Daily Basis')
plt.legend(title='Topics', loc='upper right')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Plot a single topic's occurrence on a daily basis using a line graph
topic_to_plot = 0  # Change this to the topic index you want to plot
topic_label = daily_topics_df.columns[topic_to_plot]

plt.figure(figsize=(10, 6))
plt.plot(daily_topics_df.index, daily_topics_df[topic_label])
plt.xlabel('Time')
plt.ylabel(f'Topic {topic_label} Probability')
plt.title(f'Topic {topic_label} Occurrence on a Daily Basis')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

#Time series graph of each topics per daily basis

In [ ]:


# Create a separate plot for each the topic
num_topics = daily_topics_df.shape[1]
fig, axs = plt.subplots(num_topics, 1, figsize=(10, 5*num_topics), sharex=True)

# Plot each topic's occurrence on a daily basis using a line graph
for i, topic in enumerate(daily_topics_df.columns):
    axs[i].plot(daily_topics_df.index, daily_topics_df[topic])
    axs[i].set_ylabel(f'Topic {topic} Probability')
    axs[i].grid(True)
    axs[i].set_title(f'Topic {topic}')

plt.xlabel('Time')
plt.suptitle('Topic Occurrence on a Daily Basis', fontsize=16)
plt.xticks(rotation=45)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust the subplot layout
plt.show()

#Granger causality

If the p-value is less than the chosen significance level (e.g., 0.05), it indicates that there is strong evidence to reject the null hypothesis, meaning that there is Granger causality from the first topic to the second topic.

In [ ]:
import warnings

# Ignore a specific warning by category
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [ ]:
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import grangercausalitytests
# Find Granger causality of each topic with other topics
num_topics = daily_topics_df.shape[1]
granger_causality_results = {}

for topic in daily_topics_df.columns:
    granger_causality_results[topic] = {}
    for other_topic in daily_topics_df.columns:
        if topic != other_topic:
            # Combine the two topics' data to perform the Granger causality test
            combined_data = pd.concat([daily_topics_df[topic], daily_topics_df[other_topic]], axis=1)
            results = grangercausalitytests(combined_data, maxlag=5, verbose=False)

            # Get the p-value for the Granger causality test
            p_value = results[5][0]['ssr_ftest'][1]
            granger_causality_results[topic][other_topic] = p_value

# Print the Granger causality results
for topic in granger_causality_results:
    for other_topic, p_value in granger_causality_results[topic].items():
        print(f"Granger causality: {topic} -> {other_topic}, p-value: {p_value}")

In [ ]:
import seaborn as sns
# Convert the Granger causality results to a DataFrame
granger_df = pd.DataFrame(granger_causality_results)

# Plot the Granger causality results as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(granger_df, annot=True, cmap='coolwarm', fmt=".3f")
plt.title("Granger Causality Heatmap")
plt.xlabel("Topics")
plt.ylabel("Topics")
plt.show()

In [ ]:
granger_results_text = """
Granger causality: 0 -> 1, p-value: 0.9803422495671444
Granger causality: 0 -> 2, p-value: 0.43959158595326375
Granger causality: 0 -> 3, p-value: 0.9418197702015276
Granger causality: 0 -> 4, p-value: 0.9998995106060139
Granger causality: 0 -> 5, p-value: 0.8983924581681548
Granger causality: 0 -> 6, p-value: 0.961286636922545
Granger causality: 0 -> 7, p-value: 0.9993861667584183
Granger causality: 0 -> 8, p-value: 0.80621794849973
Granger causality: 0 -> 9, p-value: 0.8412864616405612
Granger causality: 1 -> 0, p-value: 0.9587388797174856
Granger causality: 1 -> 2, p-value: 0.8074468007826063
Granger causality: 1 -> 3, p-value: 0.7924486763912888
Granger causality: 1 -> 4, p-value: 0.9998551739033269
Granger causality: 1 -> 5, p-value: 0.7988383044509652
Granger causality: 1 -> 6, p-value: 0.7396177902535095
Granger causality: 1 -> 7, p-value: 0.6531374054378156
Granger causality: 1 -> 8, p-value: 0.9904518917274396
Granger causality: 1 -> 9, p-value: 0.6131267251832068
Granger causality: 2 -> 0, p-value: 0.9982758700746208
Granger causality: 2 -> 1, p-value: 0.9998521776512177
Granger causality: 2 -> 3, p-value: 0.9099185979798545
Granger causality: 2 -> 4, p-value: 0.9999996131052048
Granger causality: 2 -> 5, p-value: 0.9898676257792559
Granger causality: 2 -> 6, p-value: 0.9551486462358112
Granger causality: 2 -> 7, p-value: 0.5541019890384038
Granger causality: 2 -> 8, p-value: 0.7386623614455949
Granger causality: 2 -> 9, p-value: 0.7590274140364133
Granger causality: 3 -> 0, p-value: 0.8192564972041879
Granger causality: 3 -> 1, p-value: 0.9783191357090125
Granger causality: 3 -> 2, p-value: 0.6482270463504463
Granger causality: 3 -> 4, p-value: 0.9985190425695399
Granger causality: 3 -> 5, p-value: 0.7786569750762601
Granger causality: 3 -> 6, p-value: 0.9252436066094694
Granger causality: 3 -> 7, p-value: 0.9978397715210413
Granger causality: 3 -> 8, p-value: 0.896737675713831
Granger causality: 3 -> 9, p-value: 0.8926610023151671
Granger causality: 4 -> 0, p-value: 0.8622168196870673
Granger causality: 4 -> 1, p-value: 0.9877906942182255
Granger causality: 4 -> 2, p-value: 0.9488288841536707
Granger causality: 4 -> 3, p-value: 0.9999593317638188
Granger causality: 4 -> 5, p-value: 0.9917585164306352
Granger causality: 4 -> 6, p-value: 0.9951656340519398
Granger causality: 4 -> 7, p-value: 0.9688739370949147
Granger causality: 4 -> 8, p-value: 0.49366227428966525
Granger causality: 4 -> 9, p-value: 0.8456799777253318
Granger causality: 5 -> 0, p-value: 0.3749833172211674
Granger causality: 5 -> 1, p-value: 0.029964986988033848
Granger causality: 5 -> 2, p-value: 0.5990093976065598
Granger causality: 5 -> 3, p-value: 0.9153164258259002
Granger causality: 5 -> 4, p-value: 0.82827389004225
Granger causality: 5 -> 6, p-value: 0.8673052050056573
Granger causality: 5 -> 7, p-value: 0.18620054639969558
Granger causality: 5 -> 8, p-value: 0.6680782903368458
Granger causality: 5 -> 9, p-value: 0.4013763184408074
Granger causality: 6 -> 0, p-value: 0.9999679641228651
Granger causality: 6 -> 1, p-value: 0.8347519086549716
Granger causality: 6 -> 2, p-value: 0.31467126289745523
Granger causality: 6 -> 3, p-value: 0.9895012767020254
Granger causality: 6 -> 4, p-value: 0.9986497616517417
Granger causality: 6 -> 5, p-value: 0.0064708969133097415
Granger causality: 6 -> 7, p-value: 0.7508354957277812
Granger causality: 6 -> 8, p-value: 0.9913918159190789
Granger causality: 6 -> 9, p-value: 0.9819548608267269
Granger causality: 7 -> 0, p-value: 0.9205853706045017
Granger causality: 7 -> 1, p-value: 0.9755590969576764
Granger causality: 7 -> 2, p-value: 0.9998150974794925
Granger causality: 7 -> 3, p-value: 0.9850975658980481
Granger causality: 7 -> 4, p-value: 0.8961323943597758
Granger causality: 7 -> 5, p-value: 0.813488495767424
Granger causality: 7 -> 6, p-value: 0.5469655320823342
Granger causality: 7 -> 8, p-value: 0.9900565911512024
Granger causality: 7 -> 9, p-value: 0.9963350515893398
Granger causality: 8 -> 0, p-value: 0.6177379207924325
Granger causality: 8 -> 1, p-value: 0.999969977058362
Granger causality: 8 -> 2, p-value: 0.9628370156176451
Granger causality: 8 -> 3, p-value: 0.9258784571536968
Granger causality: 8 -> 4, p-value: 0.7229312761917281
Granger causality: 8 -> 5, p-value: 0.6696940424384554
Granger causality: 8 -> 6, p-value: 0.9968464562868427
Granger causality: 8 -> 7, p-value: 0.8631902604516353
Granger causality: 8 -> 9, p-value: 0.9985378116400333
Granger causality: 9 -> 0, p-value: 0.8617460971795711
Granger causality: 9 -> 1, p-value: 0.9637035979787414
Granger causality: 9 -> 2, p-value: 0.08374679077256092
Granger causality: 9 -> 3, p-value: 0.2681487155573264
Granger causality: 9 -> 4, p-value: 0.9962438257260344
Granger causality: 9 -> 5, p-value: 0.03699353673995106
Granger causality: 9 -> 6, p-value: 0.17926314114714706
Granger causality: 9 -> 7, p-value: 0.6177787840724962
Granger causality: 9 -> 8, p-value: 0.9968418937202062

"""

# Split the text into lines and iterate through each line
granger_results_lines = granger_results_text.strip().split('\n')
granger_causality_results = []

for line in granger_results_lines:
    # Extract Granger causality and p-value using string manipulation
    parts = line.split(',')
    granger_causality = parts[0].split(':')[-1].strip()
    p_value = float(parts[1].split(':')[1])

    # Append the results to the list
    granger_causality_results.append((granger_causality, p_value))

# Print the extracted results
for result in granger_causality_results:
    print("Granger Causality:", result[0])
    print("P-Value:", result[1])
    print()

#Topic Evaluation

In [ ]:
import gensim
from gensim import corpora
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.models import TfidfModel

# Calculate perplexity
perplexity = lda_model.log_perplexity(corpus)
print(f"Perplexity: {perplexity}")

In [ ]:
#all_words
# Calculate coherence score
coherence_model = CoherenceModel(model=lda_model, texts=all_words, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")

In [ ]:
granger_causality_results = [
    (0, 1, 0.978),
    (0, 2, 0.245),
    (1, 3, 0.025),
    (2, 4, 0.002),
    (3, 5, 0.01),
    (1, 5, 0.001),
    (2, 8, 0.364),
    (2, 5, 0.435),
    (5, 6, 0.000),
    (7, 5, 0.023),
    (8, 4, 0.411),
    (0, 5, 0.099),
    (9, 2, 0.042),
    (9,3, 0.251),
    (9,5, 0.007),
    (9, 6, 0.111)
]

In [ ]:
# Define the threshold p-value below which we consider a significant causal relationship
threshold_p_value = 0.05

In [ ]:
# Filter the results to get pairs of topics with low p-values
significant_pairs = [(topic1, topic2) for topic1, topic2, p_value in granger_causality_results if p_value < threshold_p_value]


In [ ]:
# Print the significant pairs
print("Pairs of topics with significant Granger causality:")
for topic1, topic2 in significant_pairs:
    print(f"Topic {topic1} -> Topic {topic2}")

In [ ]:
# Sample data: granger causality results (topic index, p-value)
granger_causality_result = [
    (1, 0.97818759890553),
    (2, 0.002),
    (3, 0.01),
    ( 5, 0.01),
    (5, 0.001),
    (2, 0.042),
    (9, 0.251),
    (9, 0.007),
    (6, 0.111)


]

In [ ]:
causative_topics = []
for topic, p_value in granger_causality_result:
    if p_value <  threshold_p_value:
        causative_topics.append(topic)

print("Causative Topics:", causative_topics)

#sentiment analysis

> This section of sentiment analysis applied to topics derived from a Latent Dirichlet Allocation (LDA) model. This code snippet processes a collection of documents, calculates their sentiment scores using the VADER sentiment analysis tool, and then associates these sentiment scores with the dominant topic assigned to each document by the LDA model. Finally, it calculates and prints the average sentiment scores for each topic.

*   initializing an empty dictionary
*   obtains the topic probabilities for the document using the LDA model
*   calculates the average sentiment scores for each topic
*   List item






In [ ]:
# Download the VADER lexicon (only need to run once)
nltk.download('vader_lexicon')

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from gensim import corpora, models
from pprint import pprint

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()


In [ ]:
sentiments = []
for doc in data['Description']:
    sentiment_score = sia.polarity_scores(doc)
    sentiments.append(sentiment_score)


In [ ]:
# Print topics and sentiments
pprint(lda_model.print_topics())
for i, sentiment in enumerate(sentiments):
    print(f"Document {i + 1} - Sentiment: {sentiment}")

In [ ]:
granger_caused_topics = [(1, 3), (2, 4), (3,5),(1,5),(5,6),(7,5),(9,2),(9,5)]

In [ ]:
documents_by_topic = {topic_id: [] for topic_id in range(num_topics)}


for idx, doc_bow in enumerate(corpus):
    topic_probs = lda_model.get_document_topics(doc_bow)
    dominant_topic = max(topic_probs, key=lambda x: x[1])[0]
    documents_by_topic[dominant_topic].append(sentiments[idx])

pprint(documents_by_topic)

# Calculate sentiment scores for each topic
average_sentiments_by_topic = {}

for topic_id, doc_sentiments in documents_by_topic.items():
    avg_sentiment = {
        'neg': sum(s['neg'] for s in doc_sentiments) / len(doc_sentiments),
        'neu': sum(s['neu'] for s in doc_sentiments) / len(doc_sentiments),
        'pos': sum(s['pos'] for s in doc_sentiments) / len(doc_sentiments),
        'compound': sum(s['compound'] for s in doc_sentiments) / len(doc_sentiments)
    }
    average_sentiments_by_topic[topic_id] = avg_sentiment

# Print average sentiment scores for each topic
for topic_id, sentiment_scores in average_sentiments_by_topic.items():
    print(f"Topic {topic_id} - Average Sentiment Scores: Negative={sentiment_scores['neg']:.2f}, Neutral={sentiment_scores['neu']:.2f}, Positive={sentiment_scores['pos']:.2f}, Compound={sentiment_scores['compound']:.2f}")

In [ ]:
import matplotlib.pyplot as plt

# Extract sentiment scores for plotting
topics = []
neg_scores = []
neu_scores = []
pos_scores = []
compound_scores = []

for topic_id, sentiment_scores in average_sentiments_by_topic.items():
    topics.append(topic_id)
    neg_scores.append(sentiment_scores['neg'])
    neu_scores.append(sentiment_scores['neu'])
    pos_scores.append(sentiment_scores['pos'])
    compound_scores.append(sentiment_scores['compound'])

# Create a bar chart
fig, ax = plt.subplots(figsize=(10, 6))
bar_width = 0.2
bar_positions = [i for i in range(len(topics))]

ax.bar(bar_positions, neg_scores, width=bar_width, label='Negative')
ax.bar([pos + bar_width for pos in bar_positions], neu_scores, width=bar_width, label='Neutral')
ax.bar([pos + bar_width*2 for pos in bar_positions], pos_scores, width=bar_width, label='Positive')
ax.bar([pos + bar_width*3 for pos in bar_positions], compound_scores, width=bar_width, label='Compound')

ax.set_xlabel('Topics')
ax.set_ylabel('Sentiment Scores')
ax.set_title('Average Sentiment Scores by Topic')
ax.set_xticks([pos + bar_width*1.5 for pos in bar_positions])
ax.set_xticklabels(topics)
ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Sample Granger causality test results (replace this with your actual data)
granger_results = [
    ("0","1", 0.8182212917096419),
 ("0","2", 0.859238885348441),
 ("0", "3", 0.9294478879425941),
 ("0 ", "4",0.8832257118328085),
 ("0" ,"5", 0.7592846829068038),
 ("0", "6", 0.8483221742103257),
 ("0","7",  0.6791543256726184),
 ("0","8",  0.864711095809331),
 ("0","9",  0.8697509874387972),
 ("0","10",  0.9450324567000928),
 ("0","11",  0.8201100868419176),
 ("0","12",  0.9874761074941787),
 ("0","13",  0.9786278293045206),
 ("0","14",  0.6754253091574455),
("1","0",  0.7014741987045657),
("1","2",  0.7650042859777991),
 ("1","3",  0.9568067977978505),
 ("1","4",  0.8348775170644409),
("1","5",  0.7964192722360834),
 ("1","6",  0.7551591588788338),
 ("1","7",  0.5931320414012647),
 ("1","8",  0.8674589592180884),
 ("1","9",  0.7585673007026967),
 ("1","10",  0.912185186578581),
 ("1","11",  0.8193778675266501),
 ("1","12",  0.5747820924120519),
 ("1","13",  0.22029159351494226),
 ("1","14",  0.61453113073865),
 ("2","0",  0.984632107833643),
 ("2","1",  0.1908013237404049),
 ("2","3",  0.8629124218490352),
 ("2","4",  0.9707649055341359),
 ("2","5",  0.5233911272279366),
 ("2","6",  0.9993893485437106),
 ("2","7",  0.12175593328542862),
 ("2","8",  0.9930475017856696),
 ("2","9",  0.9994739545393406),
 ("2","10",  0.3664215613399756),
 ("2","11",  0.9425938044774969),
 ("2","12",  0.9929423142072352),
 ("2", "13",  0.9521607751600352),
 ("2","14",  0.033406257640400776),
 ("3","0",  0.30282122297149405),
 ("3","1",  0.47707501284509457),
 ( "3","2",  0.08323876979210455),
 ("3","4",  0.09134379310918268),
 ("3","5",  0.018497448757266492),
 ("3","6",  0.04959450772901254),
 ("3","7",  0.05228737581244436),
 ("3","8",  0.026470060213887377),
 ("3","9",  0.08670650365020592),
 ("3","10",  0.787960983465273),
 ("3","11",  0.0223573903476893),
 ( "3","12",  0.5266076041919772),
 ("3","13",  0.9656468863135506),
 ("3","14",  0.02928970063719082),
 ("4","0",  0.9964955849366695),
 ("4","1",  0.02518776526866879),
 ("4"," 2",  0.9321983563783551),
 ("4","3",  0.9788962807436924),
 ("4","5",  0.24635003196058652),
 ("4","6",  0.9096713982652356),
 ("4","7",  0.5389036219161133),
 ("4","8",  0.8869141019642518),
 ("4","9",  0.9998633093050577),
 ("4","10",  0.42281449546093675),
 ("4","11",  0.25386577203104654),
 ("4","12",  0.999242011556494),
 ("4","13",  0.9527416038084321),
 ("4","14",  0.0747377535280083),
 ("5","0",  0.9889204921407703),
 ("5","1",  0.5950949141134134),
 ("5","2",  0.9999175913282868),
 ("5","3",  0.9650805603329424),
 ("5","4",  0.42377821400109345),
 ("5","6",  0.9428600007086108),
 ("5","7",  0.4973226307961759),
 ("5","8",  0.8184996623346206),
 ("5","9",  0.8153166826618258),
 ("5","10",  0.7104164103257911),
 ("5","11",  0.4241910334051227),
 ("5 ","12",  0.9964560163124416),
 ("5","13",  0.15714802929173544),
 ("5","14",  0.04081711603175872),
 ("6","0",  0.9608464933262989),
 ("6","1",  0.05124658891721602),
 ("6","2",  0.9968992227362135),
 ("6","3",  0.9994365276205066),
 ("6","4",  0.9565916531450354),
 ("6","5",  0.17281108423203115),
 ("6","7",  0.3971733190905111),
 ("6","8",  0.9934205708393237),
 ("6","9",  0.9449597896305434),
 ("6","10",  0.8766209160365309),
 ("6","11",  0.6874051235044847),
 ("6","12",  0.929315162203687),
 ("6","13",  0.934729029345977),
( "6","14",  0.019524532128520053),
 ("7" ,"0",  0.9437651214881536),
 ("7" ,"1",  0.9975015833071169),
 ("7" ,"2",  0.9996595427823891),
 ("7" ,"3",  0.9980566795888703),
 ("7" ,"4",  0.995914479139652),
 ("7" ,"5",  0.997948111730317),
 ("7", "6",  0.9914471097239049),
 ("7", "8",  0.999988979389145),
 ("7", "9",  0.9995603516584118),
 ("7", "10",  0.9999308410653376),
 ("7", "11",  0.9674522529968232),
 ("7", "12",  0.9827801352978147),
 ("7", "13",  0.49110056725683293),
 ("7", "14",  0.9923670300886736),
 ("8", "0",  0.9647625319317614),
 ("8", "1",  0.12255349070520638),
 ("8", "2",  0.8932337729011303),
 ("8", "3",  0.47075426713722446),
 ("8", "4",  0.9881327308831892),
 ("8", " 5",  0.3683499713011023),
 ("8", " 6",  0.9369474874591831),
 ("8", " 7",  0.3774173556217091),
 ("8", " 9",  0.9980234466564817),
 ("8", " 10",  0.06300006500962309),
 ("8", " 11",  0.7036812215658024),
 ("8", " 12",  0.9993208877795635),
 ("8", " 13",  0.9387412211627348),
 ("8", " 14",  0.08066608395305058),
 ("9", " 0",  0.9930222786635862),
 ("9", " 1",  0.09596530339272911),
 ("9", " 2",  0.9623074309205067),
 ("9", " 3",  0.9963500669615751),
 ("9", " 4",  0.9753884448474638),
 ("9", " 5",  0.044423645358688715),
 ("9", " 6",  0.874169670826217),
 ("9", " 7",  0.337096953678197),
 ("9", " 8",  0.9649657023086383),
 ("9", " 10",  0.7608601661265086),
 ("9", " 11",  0.2880739522237003),
 ("9", " 12",  0.9777400780576486),
 ("9", " 13",  0.964858848933872),
 ("9", " 14",  0.020712650960424628),
 ("10", " 0",  0.860685835602545),
 ("10"," 1",  0.5054665216487315),
 ("10"," 2",  0.38042876586668367),
 ("10"," 3",  0.99467312042265),
 ("10"," 4",  0.31238339331058945),
 ("10"," 5",  0.024939692668784295),
 ("10"," 6",  0.2233802286245364),
 ("10"," 7",  0.17695967274830404),
 ("10"," 9",  0.16308180853388457),
 ("10"," 11",  0.05118342582561763),
 ("10"," 12",  0.9510905311451217),
 ("10"," 13",  0.9953554695970177),
 ("10"," 14",  0.049754704148925244),
 ("11"," 0",  0.9957781673284696),
 ("11"," 1",  0.06520518997199756),
 ("11"," 2",  0.9958658674217032),
 ("11"," 3",  0.9999727332273105),
 ("11"," 4",  0.8143517452710718),
 ("11"," 5",  0.1936273689980025),
 ("11"," 6",  0.9959506286315951),
 ("11", "7",  0.6238605986542511),
 ("11","8",  0.9997032759910062),
 ("11","9",  0.9326089592664748),
 ("11","10",  0.9593930931996058),
 ("11"," 12",  0.9980214120376552),
 ("11"," 13",  0.5758180167475169),
 ("11"," 14",  0.028115892435206322),
 ("12"," 0",  0.9475317182836516),
 ("12"," 1",  0.2993103990750409),
 ("12"," 2",  0.6280936855525052),
 ("12"," 3",  0.90687979024156),
 ("12"," 4",  0.7062733938084178),
 ("12"," 5",  0.41915998536217663),
 ("12"," 6",  0.5041605662053193),
 ("12"," 7",  0.3029120537698463),
 ("12"," 8",  0.6565827433088094),
 ("12"," 9",  0.7142474174415682),
 ("12"," 10",  0.8889869221339103),
 ("12"," 11",  0.45514920943221937),
 ("12"," 13",  0.9776910734751009),
 ("12"," 14",  0.3004753043111568),
 ("13"," 0",  0.7993643677452575),
 ("13"," 1",  0.4902316991002937),
 ("13"," 2",  0.9899368793451373),
 ("13"," 3",  0.7962179842428099),
 ("13"," 4",  0.9882332416045384),
 ("13"," 5",  0.9573003035922755),
 ("13"," 6",  0.9826175040279106),
 ("13"," 7",  0.9854076951062588),
 ("13"," 8",  0.9659018033204261),
 ("13"," 9",  0.9908561655921032),
 ("13"," 10",  0.9806778069462523),
 ("13"," 11",  0.9796043081037883),
 ("13"," 12",  0.9532638389692016),
 ("13"," 14",  0.9352683527711229),
 ("14"," 0",  0.9092397348789304),
 ("14"," 1",  0.9981757603545912),
 ("14"," 2",  0.9998408884195102),
 ("14"," 3",  0.9999615015368093),
 ("14"," 4",  0.9193580679323445),
 ("14"," 5",  0.5856036416060451),
 ("14"," 6",  0.9838376425026647),
 ("14"," 7",  0.8851982555049578),
 ("14"," 8",  0.988062031186449),
 ("14"," 9",  0.984010101579031),
 ("14"," 10",  0.9985524968901105),
 ("14"," 11",  0.6338315813710871),
 ("14"," 12",  0.8734660483650705),
 ("14"," 13",  0.024984933055518594)
]

# Threshold for statistical significance
threshold = 0.05

# Extract pairs of topics with p-values less than or equal to the threshold
significant_pairs = [(cause, effect) for cause, effect, p_value in granger_results if p_value <= threshold]

# Print the significant pairs
for cause, effect in significant_pairs:
    print(f"Significant Granger causality: {cause} -> {effect}")